In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import os

def process_merged_cells(ws):
    """合并单元格映射生成器"""
    merged_map = {}
    for merged_range in ws.merged_cells.ranges:
        min_row = merged_range.min_row
        min_col = merged_range.min_col
        main_value = ws.cell(row=min_row, column=min_col).value
        # 记录所有被合并单元格的坐标
        for row in range(merged_range.min_row, merged_range.max_row + 1):
            for col in range(merged_range.min_col, merged_range.max_col + 1):
                cell_coord = f"{get_column_letter(col)}{row}"
                merged_map[cell_coord] = main_value
    return merged_map

def convert_excel_ultimate(input_path):
    try:
        wb = load_workbook(input_path)
        writer = pd.ExcelWriter('Final_Output.xlsx', engine='openpyxl')
        
        # ========== 通用处理函数 ==========
        def process_sheet(ws, config):
            """通用表单处理器"""
            merged_map = process_merged_cells(ws)
            sheet_data = []
            
            for col in range(config['start_col'], ws.max_column + 1):
                col_letter = get_column_letter(col)
                
                # 获取产品名称
                product_cell = f"{col_letter}{config['title_row']}"
                product_name = merged_map.get(product_cell, ws[product_cell].value)
                if not product_name:
                    continue
                
                # 提取参数
                params = []
                for row in range(config['param_start'], config['param_end'] + 1):
                    param_name = ws.cell(row=row, column=1).value
                    param_value = merged_map.get(
                        f"{col_letter}{row}",
                        ws.cell(row=row, column=col).value
                    )
                    if param_name and param_value:
                        params.append(f"{param_name}:{param_value}")
                
                # 提取产品说明
                desc = ''
                if 'desc_row' in config:
                    desc_cell = f"{col_letter}{config['desc_row']}"
                    desc = merged_map.get(desc_cell, ws[desc_cell].value) or ''
                
                sheet_data.append({
                    '产品名称': product_name,
                    '分类': config['sheet_name'],
                    '性能参数': '\n'.join(params),
                    '产品说明': desc
                })
            
            return sheet_data

        # ========== E-Motorcycle处理 ==========
        motor_config = {
            'sheet_name': 'E-Motorcycle',
            'title_row': 2,      # 产品名称在第2行
            'start_col': 3,      # 从C列开始
            'param_start': 4,    # 参数起始行
            'param_end': 20,     # 参数结束行
            'desc_row': 23       # 说明在第23行
        }
        motor_data = process_sheet(wb['E-Motorcycle'], motor_config)
        pd.DataFrame(motor_data).to_excel(writer, motor_config['sheet_name'], index=False)

        # ========== E-Scooter处理 ==========
        scooter_config = {
            'sheet_name': 'E-Scooter',
            'title_row': 2,      # 产品名称在第2行
            'start_col': 3,      # 从C列开始
            'param_start': 4,    # 参数起始行
            'param_end': 10      # 参数结束行
        }
        scooter_data = process_sheet(wb['E-Scooter'], scooter_config)
        pd.DataFrame(scooter_data).to_excel(writer, scooter_config['sheet_name'], index=False)

        # ========== E-Bike处理 ==========
        bike_config = {
            'sheet_name': 'E-Bike',
            'title_row': 2,      # 产品名称在第2行
            'start_col': 3,      # 从C列开始
            'param_start': 4,    # 参数起始行
            'param_end': 24      # 参数结束行
        }
        bike_data = process_sheet(wb['E-Bike'], bike_config)
        pd.DataFrame(bike_data).to_excel(writer, bike_config['sheet_name'], index=False)

        # ========== E-Tricycle特殊处理 ==========
        ws_tri = wb['E-Tricycle']
        tri_merged = process_merged_cells(ws_tri)
        tri_data = []
        
        # 定义两个产品区块
        sections = [
            {'start_row': 1, 'end_row': 1, 'param_end': 7, 'type': 'E-Leisure Tricycle'},
            {'start_row': 8, 'end_row': 8, 'param_end': 15, 'type': 'E-Cargo Tricycle'}
        ]
        
        for section in sections:
            for col in range(3, ws_tri.max_column + 1):
                col_letter = get_column_letter(col)
                
                # 获取产品名称
                product_cell = f"{col_letter}{section['start_row']}"
                product_name = tri_merged.get(product_cell, ws_tri[product_cell].value)
                if not product_name:
                    continue
                
                # 提取参数
                params = []
                for row in range(section['start_row'] + 1, section['param_end'] + 1):
                    param_name = ws_tri.cell(row=row, column=1).value
                    param_value = tri_merged.get(
                        f"{col_letter}{row}",
                        ws_tri.cell(row=row, column=col).value
                    )
                    if param_name and param_value:
                        params.append(f"{param_name}:{param_value}")
                
                tri_data.append({
                    '产品名称': product_name,
                    '分类': 'E-Tricycle',
                    '性能参数': '\n'.join(params),
                    '产品说明': section['type']
                })
        
        pd.DataFrame(tri_data).to_excel(writer, 'E-Tricycle', index=False)

        writer.close()
        
        # 结果验证
        if os.path.exists('Final_Output.xlsx') and os.path.getsize('Final_Output.xlsx') > 1024:
            print("生成成功！文件大小：", os.path.getsize('Final_Output.xlsx'), "字节")
            return True
        return False

    except Exception as e:
        print(f"生成失败: {str(e)}")
        return False

# 执行转换
if convert_excel_ultimate('AI机器人产品素材提供(2).xlsx'):
    print("请检查 Final_Output.xlsx")
else:
    print("转换失败，请检查输入文件")


文件生成成功！
请检查 output_终极版.xlsx


C:\Users\lilih\AppData\Local\Temp\ipykernel_20300\2617167154.py:61: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  pd.DataFrame(motor_data).to_excel(writer, 'E-Motorcycle', index=False)
C:\Users\lilih\AppData\Local\Temp\ipykernel_20300\2617167154.py:104: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  pd.DataFrame(tri_data).to_excel(writer, 'E-Tricycle', index=False)


: 